In [1]:
import pandas as pd
import numpy as np

In [54]:
houses = pd.read_csv("https://raw.githubusercontent.com/Johnle3/MLHousingPrices/main/redfin_2022-06-06-11-05-25.csv")
# Remove unnecessary columns
houses = houses.drop('FAVORITE', axis=1)
houses = houses.drop(['SALE TYPE', 'SOLD DATE','ADDRESS','HOA/MONTH', 'NEXT OPEN HOUSE START TIME','NEXT OPEN HOUSE END TIME','INTERESTED','URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)','SOURCE', 'MLS#','STATUS','LOCATION','DAYS ON MARKET'], axis = 1)

#Subset to only Alexandria and without Vacant Lot
houses = houses[houses['CITY']!= 'Arlington']
houses = houses[houses['PROPERTY TYPE'] != "Vacant Land"]
houses = houses[houses['PROPERTY TYPE'] != "Condo/Co-op"]
# Drop null values
#houses = houses.dropna()
houses = houses.dropna(subset=['LOT SIZE', 'YEAR BUILT'], how='any')
houses.head()

,PROPERTY TYPE,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,SQUARE FEET,LOT SIZE,YEAR BUILT,$/SQUARE FEET,LATITUDE,LONGITUDE
0,Single Family Residential,Alexandria,VA,22308,45000000,7.0,10.0,25500.0,435600.0,2023.0,1765.0,38.739577,-77.045973
1,Single Family Residential,Alexandria,VA,22314,5400000,6.0,5.0,8145.0,21153.0,1795.0,663.0,38.809834,-77.044992
4,Single Family Residential,Alexandria,VA,22309,3650000,4.0,5.0,5602.0,21972.0,2010.0,652.0,38.699743,-77.116651
7,Single Family Residential,Alexandria,VA,22309,3250000,6.0,6.5,5590.0,105654.0,2021.0,581.0,38.703304,-77.094285
8,Single Family Residential,Alexandria,VA,22314,3195000,4.0,3.5,4122.0,3116.0,1800.0,775.0,38.803387,-77.044415


In [55]:
# null values
houses.isna().sum()

PROPERTY TYPE         0
CITY                  0
STATE OR PROVINCE     0
ZIP OR POSTAL CODE    0
PRICE                 0
BEDS                  3
BATHS                 3
SQUARE FEET           0
LOT SIZE              0
YEAR BUILT            0
$/SQUARE FEET         0
LATITUDE              0
LONGITUDE             0
dtype: int64

In [56]:
houses[houses['BATHS'].isna()] # bed/bath values may be imputed: KNNeighbor

,PROPERTY TYPE,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,SQUARE FEET,LOT SIZE,YEAR BUILT,$/SQUARE FEET,LATITUDE,LONGITUDE
41,Multi-Family (2-4 Unit),Alexandria,VA,22314,1749999,NaN,NaN,1892.0,1408.0,1880.0,925.0,38.804275,-77.050697
42,Multi-Family (2-4 Unit),Alexandria,VA,22301,1749999,NaN,NaN,2618.0,7600.0,1925.0,668.0,38.820500,-77.056514
92,Multi-Family (2-4 Unit),Alexandria,VA,22314,1450000,NaN,NaN,3500.0,7306.0,1960.0,414.0,38.796159,-77.051167


In [57]:
houses[houses['LOT SIZE'].isna()] #Drop? or maybe fill in with something? KNN?

,PROPERTY TYPE,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,SQUARE FEET,LOT SIZE,YEAR BUILT,$/SQUARE FEET,LATITUDE,LONGITUDE


In [58]:
houses[houses['YEAR BUILT'].isna()]

,PROPERTY TYPE,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,SQUARE FEET,LOT SIZE,YEAR BUILT,$/SQUARE FEET,LATITUDE,LONGITUDE


In [59]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=18)

In [60]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='mean') #could also do 'mean', 'median', (strategy='constant', 1)

In [61]:
houses['PROPERTY TYPE'].value_counts() # --> dummy vars

Single Family Residential    184
Townhouse                     76
Multi-Family (2-4 Unit)        3
Name: PROPERTY TYPE, dtype: int64

In [62]:
import sklearn
from sklearn import linear_model
X_train = imputer.fit_transform(X_train)
X_test = imputer.transform(X_test)
X_test[:1]

array([[3.000e+00, 1.500e+00, 1.405e+03, 3.450e+03, 1.930e+03]])

In [63]:
model = linear_model.Lasso(alpha=1.0)

In [64]:
from numpy import arange
from numpy import mean
from numpy import std
from numpy import absolute
from pandas import read_csv
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV

In [65]:
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)

In [66]:
X,y= houses[['BEDS','BATHS','SQUARE FEET', 'LOT SIZE','YEAR BUILT']], houses[['PRICE']]
scores = cross_val_score(model, X, y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
scores = absolute(scores)
print('Mean MAE: %.3f (%.3f)' % (mean(scores), std(scores)))

Mean MAE: nan (nan)


In [67]:
grid = dict()
grid['alpha'] = arange(0, 1, 0.01)

In [68]:
search = GridSearchCV(model, grid, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)

In [69]:
from pandas import read_csv
from sklearn.linear_model import Lasso
# load the dataset
# define model
model = Lasso(alpha=1.0)
# fit model
model.fit(X, y)
# define new data
row = [2,2,10000,2000,2000]
# make a prediction
yhat = model.predict([row])
# summarize prediction
print('Predicted: %.3f' % yhat)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [ ]:
row = [2,2,10000,2000,2020]
# make a prediction
yhat = model.predict([row])
print('Predicted: %.3f' % yhat)